In [2]:
import asyncio
from Secrets import secrets
from azure.identity import ClientSecretCredential
from msgraph import GraphServiceClient
import requests
from kiota_abstractions.api_error import APIError
import msal

# Get token
client_secret = secrets.get('CLIENT_SECRET')
client_id = secrets.get('CLIENT_ID')
tenant_id =  secrets.get('TENANT_ID')
authority= 'https://login.microsoftonline.com/{}'.format(secrets.get('TENANT_ID'))
scope = ['https://graph.microsoft.com/.default']

In [3]:
# Create an MSAL instance providing the client_id, authority and client_credential parameters
client = msal.ConfidentialClientApplication(client_id, authority=authority, client_credential=client_secret)

# First, try to lookup an access token in cache
token_result = client.acquire_token_silent(scope, account=None)
access_token=''

# If the token is available in cache, save it to a variable
if token_result:
    access_token = 'Bearer ' + token_result['access_token']
    print('Access token was loaded from cache')

# If the token is not available in cache, acquire a new one from Azure AD and save it to a variable
if not token_result:
    token_result = client.acquire_token_for_client(scopes=scope)
    access_token = 'Bearer ' + token_result['access_token']
    print('New access token was acquired from Azure AD')

print(access_token)

New access token was acquired from Azure AD
Bearer eyJ0eXAiOiJKV1QiLCJub25jZSI6InlfOXc4cEZhdmQ5U0V1dUl0eHhINjVYRnpucHc4cFFCR2YwZnJ2S0FlLVUiLCJhbGciOiJSUzI1NiIsIng1dCI6IlhSdmtvOFA3QTNVYVdTblU3Yk05blQwTWpoQSIsImtpZCI6IlhSdmtvOFA3QTNVYVdTblU3Yk05blQwTWpoQSJ9.eyJhdWQiOiJodHRwczovL2dyYXBoLm1pY3Jvc29mdC5jb20iLCJpc3MiOiJodHRwczovL3N0cy53aW5kb3dzLm5ldC8wOGIwNDk2Yi01NWYwLTRkOTQtYWFjYy03NGE3NWQ2YjgyOWYvIiwiaWF0IjoxNzExNTUxODExLCJuYmYiOjE3MTE1NTE4MTEsImV4cCI6MTcxMTU1NTcxMSwiYWlvIjoiRTJOZ1lIZzE3Y0hjTHAwTDd0V2xYK3YrTUwwWEJ3QT0iLCJhcHBfZGlzcGxheW5hbWUiOiJTaGFyZXBvaW50IiwiYXBwaWQiOiI4YjQxNTAxZS1mZDBmLTQ5YjYtODNlOS01Y2RlOTY0ZTY3ZjgiLCJhcHBpZGFjciI6IjEiLCJpZHAiOiJodHRwczovL3N0cy53aW5kb3dzLm5ldC8wOGIwNDk2Yi01NWYwLTRkOTQtYWFjYy03NGE3NWQ2YjgyOWYvIiwiaWR0eXAiOiJhcHAiLCJvaWQiOiI2NTEwOThlMC00ZTY2LTQ2Y2MtOGZhZS1hNmM5NDM3YWE1MTIiLCJyaCI6IjAuQWE4QWEwbXdDUEJWbEUycXpIU25YV3VDbndNQUFBQUFBQUFBd0FBQUFBQUFBQUN2QUFBLiIsInJvbGVzIjpbIk1haWwuUmVhZFdyaXRlIiwiRGV2aWNlLlJlYWRXcml0ZS5BbGwiLCJEZXZpY2VMb2NhbENyZWRlbnRpYWwuUmVh

In [8]:
print(access_token)

Bearer eyJ0eXAiOiJKV1QiLCJub25jZSI6InlfOXc4cEZhdmQ5U0V1dUl0eHhINjVYRnpucHc4cFFCR2YwZnJ2S0FlLVUiLCJhbGciOiJSUzI1NiIsIng1dCI6IlhSdmtvOFA3QTNVYVdTblU3Yk05blQwTWpoQSIsImtpZCI6IlhSdmtvOFA3QTNVYVdTblU3Yk05blQwTWpoQSJ9.eyJhdWQiOiJodHRwczovL2dyYXBoLm1pY3Jvc29mdC5jb20iLCJpc3MiOiJodHRwczovL3N0cy53aW5kb3dzLm5ldC8wOGIwNDk2Yi01NWYwLTRkOTQtYWFjYy03NGE3NWQ2YjgyOWYvIiwiaWF0IjoxNzExNTUxODExLCJuYmYiOjE3MTE1NTE4MTEsImV4cCI6MTcxMTU1NTcxMSwiYWlvIjoiRTJOZ1lIZzE3Y0hjTHAwTDd0V2xYK3YrTUwwWEJ3QT0iLCJhcHBfZGlzcGxheW5hbWUiOiJTaGFyZXBvaW50IiwiYXBwaWQiOiI4YjQxNTAxZS1mZDBmLTQ5YjYtODNlOS01Y2RlOTY0ZTY3ZjgiLCJhcHBpZGFjciI6IjEiLCJpZHAiOiJodHRwczovL3N0cy53aW5kb3dzLm5ldC8wOGIwNDk2Yi01NWYwLTRkOTQtYWFjYy03NGE3NWQ2YjgyOWYvIiwiaWR0eXAiOiJhcHAiLCJvaWQiOiI2NTEwOThlMC00ZTY2LTQ2Y2MtOGZhZS1hNmM5NDM3YWE1MTIiLCJyaCI6IjAuQWE4QWEwbXdDUEJWbEUycXpIU25YV3VDbndNQUFBQUFBQUFBd0FBQUFBQUFBQUN2QUFBLiIsInJvbGVzIjpbIk1haWwuUmVhZFdyaXRlIiwiRGV2aWNlLlJlYWRXcml0ZS5BbGwiLCJEZXZpY2VMb2NhbENyZWRlbnRpYWwuUmVhZEJhc2ljLkFsbCIsIlNpdGVzLlJlYWRXcml0ZS5BbGwi

In [13]:
graph_results=[]
url= 'https://graph.microsoft.com/v1.0/deviceManagement/managedDevices'
headers = {
    'Authorization': access_token
}
while url:
    try:
        graph_result = requests.get(url=url, headers=headers).json()
        graph_results.extend(graph_result['value'])
        url = graph_result['@odata.nextLink']        
    except:
        break
print(graph_results)

[{'id': '072963c9-42bc-46ca-89cf-203b8ffb3e6d', 'userId': 'c19530ea-ca1e-484d-a90c-a443f28477f6', 'deviceName': 'CPC-Jcapo-XUZEE', 'managedDeviceOwnerType': 'company', 'enrolledDateTime': '2023-08-25T16:13:01Z', 'lastSyncDateTime': '2023-09-05T02:31:27Z', 'operatingSystem': 'Windows', 'complianceState': 'noncompliant', 'jailBroken': 'Unknown', 'managementAgent': 'mdm', 'osVersion': '10.0.22621.2134', 'easActivated': False, 'easDeviceId': '', 'easActivationDateTime': '0001-01-01T00:00:00Z', 'azureADRegistered': True, 'deviceEnrollmentType': 'windowsAzureADJoin', 'activationLockBypassCode': None, 'emailAddress': 'Jcapobianco@q7vf.onmicrosoft.com', 'azureADDeviceId': '072963c9-42bc-46ca-89cf-203b8ffb3e6d', 'deviceRegistrationState': 'registered', 'deviceCategoryDisplayName': 'Unknown', 'isSupervised': False, 'exchangeLastSuccessfulSyncDateTime': '0001-01-01T00:00:00Z', 'exchangeAccessState': 'none', 'exchangeAccessStateReason': 'none', 'remoteAssistanceSessionUrl': None, 'remoteAssistance